In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from keras import initializers
import shutil

In [ ]:
# !tar -xvf /kaggle/input/woof-assignmentpt2/imagewoof-320.tgz

In [ ]:

def merge_datasets(train_dir, val_dir, master_dir):
    # Ensure the master directory exists
    if not os.path.exists(master_dir):
        os.makedirs(master_dir)

    # Process each dataset directory (train, val)
    for dataset_dir in [train_dir, val_dir]:
        # List all class directories in the current dataset directory
        for class_name in os.listdir(dataset_dir):
            class_dir_path = os.path.join(dataset_dir, class_name)
            # Skip if not a directory
            if not os.path.isdir(class_dir_path):
                continue

            # Ensure corresponding class directory exists in the master directory
            master_class_dir = os.path.join(master_dir, class_name)
            if not os.path.exists(master_class_dir):
                os.makedirs(master_class_dir)

            # Copy all images from this class to the master directory
            for image_name in os.listdir(class_dir_path):
                source_image_path = os.path.join(class_dir_path, image_name)
                destination_image_path = os.path.join(master_class_dir, image_name)

                # To avoid overwriting, check if the file already exists
                if not os.path.exists(destination_image_path):
                    shutil.copy2(source_image_path, destination_image_path)
                else:
                    print(f"Skipping {destination_image_path}, already exists.")

# Define your directory paths
train_dir = '/kaggle/working/imagewoof-320/train'
val_dir = '/kaggle/working/imagewoof-320/val'
master_dir = '/kaggle/working/imagewoof-320/test'

# Merge the datasets
merge_datasets(train_dir, val_dir, master_dir)

print("Merging completed.")

In [ ]:
def count_files_in_subfolders(master_dir):
    # Dictionary to hold the count of files in each subfolder
    file_counts = {}

    # List all items in the master directory
    for subfolder in os.listdir(master_dir):
        subfolder_path = os.path.join(master_dir, subfolder)
        # Ensure the item is a directory
        if os.path.isdir(subfolder_path):
            # Count the number of files in this subfolder
            file_counts[subfolder] = len([name for name in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, name))])

    return file_counts

# # Define your master directory path
# master_dir = '/kaggle/working/imagenette2-160/test'

# Get the file counts
file_counts = count_files_in_subfolders(master_dir)

# Print the counts
for subfolder, count in file_counts.items():
    print(f"{subfolder}: {count} files")

In [ ]:
classes_to_include = ['n02087394', 'n02096294','n02099601','n02111889']
classes_to_include
#Correcting any accidental leading/trailing whitespace in class names
classes_to_include = [cls.strip() for cls in classes_to_include]

In [ ]:
def get_subdirectories(directory):
    subdirectories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return subdirectories

get_subdirectories("/kaggle/working/imagewoof-320/test")

In [ ]:


def include_subdirectories(directory, classes_to_include):
    sub_dir_list = get_subdirectories(directory)
    for c in sub_dir_list:
        # If the subdirectory is not in the list of classes to include
        if c not in classes_to_include:
            # Construct the path to the subdirectory
            dir_to_delete = os.path.join(directory, c)
            # Remove the subdirectory and all its contents
            shutil.rmtree(dir_to_delete)
            print(f"Deleted: {dir_to_delete}")
include_subdirectories('/kaggle/working/imagewoof-320/test', classes_to_include)

In [ ]:
pip install split-folders[full]

In [ ]:
import splitfolders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/kaggle/working/imagewoof-320/test", output="TrainTestValidation",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values


In [ ]:

train_dataset = image_dataset_from_directory(
"/kaggle/working/TrainTestValidation/train",
image_size=(299,299),
batch_size=32)

val_dataset = image_dataset_from_directory(
"/kaggle/working/TrainTestValidation/val",
image_size=(299,299),
batch_size=32)

test_dataset = image_dataset_from_directory(
"/kaggle/working/TrainTestValidation/test",
image_size=(299,299),
batch_size=32)

In [ ]:
for images, labels in train_dataset.take(1):
    print(images.shape)  # Should be (batch_size, 299, 299, 3)
    print(labels.shape)  # Should be (batch_size,) for sparse_categorical_crossentropy
    print(labels.dtype)  # Should be tf.int32 or tf.int64

In [ ]:
#Want to get example of each class to see what they look like
#Want to be able to resuse code after augmentation
#Write funcitons to do this
train_datasetViz = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/working/imagewoof-320/train",
    image_size=(299, 299),
    batch_size=32,
    shuffle=True,
    seed=42
)

def filter_class(images, labels, class_index):
    return tf.equal(labels, class_index)

def apply_filter(dataset, class_index):
    filtered_dataset = dataset.unbatch().filter(lambda image, label: filter_class(image, label, class_index)).batch(3)
    return filtered_dataset

def make_plot_data_set_viz(filtered_dataset, class_index):
    plt.figure(figsize=(10, 10))
    for images, labels in filtered_dataset.take(1):
        for i in range(3):  # Looping through 3 examples
            ax = plt.subplot(1, 3, i + 1)  # Adjust subplot for a row of 3 images
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f'Class {class_index}')
            plt.axis("off")
    plt.show()

def filter_and_plot_class_tfdataset(dataset, class_index):
    # Apply the filter
    filtered_dataset = apply_filter(dataset, class_index)
    # Make plot
    make_plot_data_set_viz(filtered_dataset, class_index)


In [ ]:
for x in range(6):
    filter_and_plot_class_tfdataset(train_dataset,x)

In [ ]:
from keras.applications.inception_v3 import InceptionV3

conv_base = InceptionV3(
    weights='imagenet',
    include_top=False)

for layer in conv_base.layers[:-62]: # Chollet 8.27
    layer.trainable = False

inputs = keras.Input(shape=(299,299,3))
x = layers.Rescaling(1./255)(inputs)
x = conv_base(x)
x= layers.Flatten()(x)
outputs = layers.Dense(4, activation = 'softmax')(x)

model1 = keras.Model(inputs=inputs, outputs=outputs)

model1.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = "adam",
                 metrics =["accuracy"])
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath ="InceptionV3_Finetune1.keras",
    save_best_only=True,
    monitor="val_loss"),
     keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True
    )
]

history = model1.fit(
train_dataset,
epochs=100,
validation_data=val_dataset,
callbacks = callbacks)

In [ ]:
model1.save('my_model.h5')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

val_images, val_labels = next(iter(val_dataset.unbatch().batch(len(val_dataset))))
predictions = model1.predict(val_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = val_labels.numpy()

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

# Calculate per-class accuracy
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
for idx, acc in enumerate(per_class_accuracy):
    print(f"Class {idx} accuracy: {acc*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs ,accuracy, "bo", label = "Training Accuracy")
plt.plot(epochs ,val_accuracy, "b", label = "Validation Accuracy")             
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label= "Training loss")
plt.plot(epochs, val_loss, "b", label=" Validation loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()